# Übungsklausur SWK5UE WS23/24

### Aufgabe 1a
Implementieren Sie eine Methode ```Characters```, die folgendermaßen verwendet werden kann:  
```
IEnumerable<char> chars = "Hello!".Characters();
// -> ['H','e','l','l','o','!']
```
Sorgen Sie dafür, dass die zurückgegebene Funktion erst dann evaluiert wird, wenn durch ihre Elemente iteriert wird.

In [45]:

public static IEnumerable<char> Characters(this string input)
{
	foreach (var ch in input)
	{
		yield return ch;
	}
}

var test = "Hello!".Characters();
foreach(var c in test)
{
	Console.Write(c);
}

Hello!

### Aufgabe 1b
Erweitern Sie das Programm, sodass aus ```chars``` die Sonderzeichen entfernt, und die verbleibenden Zeichen in Großbuchstaben konvertiert werden.  

```IEnumerable<char> chars = ... -> ['H','E','L','L','O']```  

Setzen Sie dafür soweit wie möglich LINQ ein. Sie können dafür die Klassenmethoden ```Char.IsLetter()``` und ```Char.ToUpper()``` verwenden.

In [46]:
var test1 = from ch in test
			where Char.IsLetter(ch)
			select Char.ToUpper(ch);

foreach(var c in test1)
{
	Console.Write(c);
}

HELLO

### Aufgabe 2a
Implementieren Sie eine Klasse ```Executor<T>``` die folgendes leistet:
- Sie stellt eine Methode  
```public Task ApplyAsync(Func<T> func, T arg);```  
zur Verfügung, an die eine Funktion von folgendem Datentyp übergeben wird:  
```delegate T Func<T>(T arg);```  
Der zeite Parameter ist ein Wert, mit der ```func``` aufgerufen werden kann.  

- ```ApplyAsync``` führt die übergebene Funktion mit dem übergebenen Argument in einer Task aus.
Verwenden Sie dazu die Methode Task.Run mit folgender Signatur:  
```public static Task<T> Run<T>(Func<T> function);```  

- Das Ergebnis des Funktionsaufrufes wird den (registrierten) Verwendern der Klasse über das Event ```ResultComputed``` mitgeteilt.  

- Wird ```ApplyAsync``` aufgerufen, während eine Berechnung im Gange ist, wird eine ```InvalidOperationException``` geworfen.  

- Geben Sie auch den Datentyp von ```ResultComputed``` an.  


In [1]:
public delegate T Func<T>(T arg);
public delegate void ResultComputedDelegate<T>(T res);

public class Executor<T>
{
	public event ResultComputedDelegate<T> ResultComputed;
	private bool isRunning = false;

	public Task ApplyAsync(Func<T> func, T arg)
	{
        if (isRunning)
        {
            throw new InvalidOperationException("A calculation is already running.");
        }

        isRunning = true;

        return Task.Run(() =>
        {
            try
            {
                ResultComputed?.Invoke(func(arg));
            }
            finally
            {
                isRunning = false;
            }
        });
	}
}


Zeigen Sie, wie man Ihre Klasse Vervendet, indem Sie asynchron die Quadratwurzel von 2 bestimmen und das Ergebnis auf der Konsole ausgeben.

In [2]:
public double SqrtFunc(double input) => Math.Sqrt(input);

Executor<double> exec = new();
exec.ResultComputed += res => Console.WriteLine($"Result: {res}");

await exec.ApplyAsync(SqrtFunc, 2);
await exec.ApplyAsync(SqrtFunc, 3);

Result: 1.4142135623730951
Result: 1.7320508075688772


### Aufgabe 2 ADO.NET
In der Tabelle ```tbl_persion (id: int, name : varchar (255), picture_url : varchar(255))``` werden personenbezogene Daten gespeichert.  
Der Zugriff auf diese Tabelle soll in einem Data Access Object (DAO) gekapselt werden.  

a) Entwerfen Sie dafür eine DAO-Schnittstelle, welche eine Methode zum Auslesen aller Personen sowie eine Methode zum Aktualisieren der Daten einer bestimmten Person zur Verfugung stellt.  
Geben Sie auch die Definition der Domänenklasse in Form eines Record-Datentyps an.  

b) lmplementieren Sie die DAO-Methode zum Auslesen der Daten aller Personen mithilfe von ADO.NET.  
Gestalten Sie den Zugriffscode moglichst Provider-unabhangig.  

Hinweise:  
• Verwenden Sie zur Ermittlung der Verbindungsparameter die unten angegebenen Hilfsmethode.  
• Die Signaturen der zur Losung dieser Aufgabe relevanten ADO.NET-Methoden sind ebenfalls
unten angeführt.  
• Die in der Übung entwickelten Unterstutzungsklassen (AdoTemplate etc.) durfen Sie
nicht verwenden.  

Hilfsmethode zum Zugriff zur Ermittlung des Connection-Strings und des Provider-Namens:

```csharp
public static class ConfigurationUtil {
    public static (string ConnectionString, string ProviderName) GetConnectionParameters();
}
```

Schnittstellen relevanter ADO.NET-Methoden:
```csharp
public class DbProviderFactories {
    public static DbProviderFactory GetFactory(string providerName);
}
public abstract class DbProviderFactory {
    public virtual DbConnection CreateConnection();
}
public abstract class DbConnection : IDisposable {
    public abstract string ConnectionString { get; set; }
    public abstract void Open();
    public abstract DbCommand CreateCommand();
}
public abstract class DbCommand : IDisposable {
    public abstract string CommandText { get; set; }
    public abstract DbDataReader ExecuteReader();
}
public abstract class DbDataReader : IDataReader, IDataRecord, IDisposable {
    public abstract bool Read();
    public abstract object this[string name] { get; }
}
```

In [1]:
public record Person(
	string Id,
	string Name,
	string PictureUrl
);

public interface IPersonDao
{
	public IEnumerable<Person> GetAllPersons();
	public bool UpdatePerson(Person p);
}


public static class ConfigurationUtil {
	public static (string ConnectionString, string ProviderName) GetConnectionParameters()
	{
		string connectionString = "Your connection string";
		string providerName = "Your provider name";

		return (connectionString, providerName);
	}
}

In [3]:
using System;
using System.Collections.Generic;
using System.Data.Common;

public class PersonDao : IPersonDao
{
	
	public IEnumerable<Person> GetAllPersons()
	{
		var (connectionString, providerName) = ConfigurationUtil.GetConnectionParameters();
		var factory = DbProviderFactories.GetFactory(providerName);

		using var connection = factory.CreateConnection();
		connection.ConnectionString = connectionString;
		connection.Open();

		using var command = connection.CreateCommand();
		command.CommandText = "SELECT Id, Name, PictureUrl FROM Persons";

		using var reader = command.ExecuteReader();
		var persons = new List<Person>();

		while (reader.Read())
		{
			var person = new Person(
				reader["Id"].ToString(),
				reader["Name"].ToString(),
				reader["PictureUrl"].ToString()
			);

			persons.Add(person);
		}

		return persons;
	}

	public bool UpdatePerson(Person p)
	{
		throw new NotImplementedException();
	}
}